## Setup

In [1]:
import os
import numpy as np
import time
import cv2
from __future__ import print_function

## Loop

In [77]:
USE_CAM = True
LOOP = True    # loop through videos
DISP_BW = False
DISP_R_CHAN = True
DISP_G_CHAN = True
DISP_B_CHAN = True
DISP_BORDER = 0

THRESH = False
THRESH_STYLE = 0
USE_ALL_CHAN = True
USE_R_CHAN = False
USE_G_CHAN = False
USE_B_CHAN = False


# select video source
if USE_CAM:
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture('/media/data/Dropbox/Git/vid-viz/data/test.mp4')
    
# get video properties
im_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
im_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
target_fps = cap.get(cv2.CAP_PROP_FPS)
tot_fr_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

fr_count = 0
while(cap.isOpened()):

    time_pre = time.time()
    
    ret, frame = cap.read()

    # use keyboard input to change effect
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('s'):
        # toggle color
        DISP_BW = not DISP_BW
    elif key == ord('r'):
        DISP_R_CHAN = not DISP_R_CHAN
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = True
        USE_G_CHAN = False
        USE_B_CHAN = False
    elif key == ord('g'):
        DISP_G_CHAN = not DISP_G_CHAN
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = False
        USE_G_CHAN = True
        USE_B_CHAN = False
    elif key == ord('b'):
        DISP_B_CHAN = not DISP_B_CHAN
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = False
        USE_G_CHAN = False
        USE_B_CHAN = True
    elif key == ord('z'):
        DISP_BORDER = (DISP_BORDER + 1) % 3
    elif key == ord('t'):
        THRESH = not THRESH
        USE_ALL_CHAN = True
        DISP_R_CHAN = True
        DISP_G_CHAN = True
        DISP_B_CHAN = True
    elif key == ord('y'):
        THRESH_STYLE = (THRESH_STYLE + 1) % 2
    elif key == ord('v'):
        USE_ALL_CHAN = True
        
        
    # implement effects
    if not THRESH:
        if not DISP_R_CHAN:
            frame[:,:,2] = 0
        if not DISP_G_CHAN:
            frame[:,:,1] = 0
        if not DISP_B_CHAN:
            frame[:,:,0] = 0
    if DISP_BW:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if DISP_BORDER == 1:
        # top, bottom, left, right
        frame = cv2.copyMakeBorder(frame, int(im_height/4), int(im_height/4),
                                          int(im_width/4), int(im_width/4),
                                          cv2.BORDER_WRAP)
    elif DISP_BORDER == 2:
        # top, bottom, left, right
        frame = cv2.copyMakeBorder(frame, int(im_height/2), 0,
                                          int(im_width/2), 0,
                                          cv2.BORDER_REFLECT)
    if THRESH:
        
        # BINARY | BINARY_INV | TRUNC | TOZERO | TOZERO_INV
        if THRESH_STYLE == 0:
            THRESH_TYPE = cv2.THRESH_BINARY
        elif THRESH_STYLE == 1:
            THRESH_TYPE = cv2.THRESH_BINARY_INV
            
        # MEAN_C | GAUSSIAN_C
        ADAPTIVE_THRESH_TYPE = cv2.ADAPTIVE_THRESH_MEAN_C
        THRESH_CEIL = 255
        THRESH_BLOCK = 15
        THRESH_C = 4
        
        if USE_ALL_CHAN:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame = cv2.adaptiveThreshold(frame, THRESH_CEIL, 
                                          ADAPTIVE_THRESH_TYPE, THRESH_TYPE, 
                                          THRESH_BLOCK, THRESH_C)
        elif USE_R_CHAN:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # zero out blue channel, threshold red channel
            frame[:,:,0] = 0
            frame[:,:,2] = cv2.adaptiveThreshold(gray_frame, THRESH_CEIL, 
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE, 
                                                 THRESH_BLOCK, THRESH_C)
        elif USE_G_CHAN:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # zero out red channel, threshold green channel
            frame[:,:,2] = 0
            frame[:,:,1] = cv2.adaptiveThreshold(gray_frame, THRESH_CEIL,
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE,
                                                 THRESH_BLOCK, THRESH_C)
        elif USE_B_CHAN:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # zero out green channel, threshold red channel
            frame[:,:,1] = 0
            frame[:,:,0] = cv2.adaptiveThreshold(gray_frame, THRESH_CEIL,
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE,
                                                 THRESH_BLOCK, THRESH_C)
    
    # display image
    cv2.imshow('frame', frame)
    
    # reset video
    fr_count += 1
    if fr_count == tot_fr_count and LOOP:
        # reset frame postion to zero
        cap.set(2,0)
        fr_count = 0
    
    # calculate, limit and output fps
    time_post = time.time()
    time_tot = time_post - time_pre
    if time_tot < 1/target_fps:
        time.sleep(1/target_fps - time_tot)
    time_post = time.time()
    time_tot = time_post - time_pre
    fps = 1.0 / time_tot
    print('\r%03i fps' % fps, end='')
    
    
cap.release()
cv2.destroyWindow('frame')

014 fps

In [28]:
flags = [i for i in dir(cv2) if i.startswith('COLOR_')]
print(flags[:2])

['COLOR_BAYER_BG2BGR', 'COLOR_BAYER_BG2BGR_EA']


In [2]:
img = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test.jpg', 0) 
img.dtype

dtype('uint8')

In [2]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'dv2' is not defined

In [4]:
whos

Variable         Type                   Data/Info
-------------------------------------------------
CAM              bool                   False
DISP_BORDER      int                    0
DISP_BW          bool                   False
DISP_B_CHAN      bool                   True
DISP_G_CHAN      bool                   True
DISP_R_CHAN      bool                   True
LOOP             bool                   True
cap              VideoCapture           <VideoCapture 0x7f158d7016b0>
cv2              module                 <module 'cv2' from '/usr/<...>.7/dist-packages/cv2.so'>
fps              float                  29.8810679709
fr_count         int                    132
frame            ndarray                616x1096x3: 2025408 elems, type `uint8`, 2025408 bytes (1 Mb)
im_height        float                  616.0
im_width         float                  1096.0
key              int                    113
np               module                 <module 'numpy' from '/us<...>ages/numpy/__